In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import *
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold

# Importar modelos e suas respectivas funções preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input as preprocess_xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_vgg19
from tensorflow.keras.applications.resnet import ResNet50, ResNet101, ResNet152, preprocess_input as preprocess_resnet
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input as preprocess_resnet_v2
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_inception_v3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input as preprocess_inception_resnet_v2
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_mobilenet_v2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201, preprocess_input as preprocess_densenet
from tensorflow.keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input as preprocess_nasnet
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7, preprocess_input as preprocess_efficientnet
# Add EfficientNetV2 imports if available in your TensorFlow version
# from tensorflow.keras.applications.efficientnet_v2 import ...

# Definição do tamanho de entrada para cada modelo de CNN
model_input_sizes = {
    'Xception': (256, 256), #1
    'VGG16': (256, 256), #2
    'VGG19': (256, 256), #3 
    'ResNet50': (256, 256), #4 
    'ResNet50V2': (256, 256), #5 
    'ResNet101': (256, 256), #6 
    'ResNet101V2': (256, 256), #7 
    'ResNet152': (256, 256), #8
    'ResNet152V2': (256, 256), #9
    'InceptionV3': (256, 256), #10 
    'InceptionResNetV2': (256, 256), #11
    'MobileNet': (256, 256), #12 
    'MobileNetV2': (256, 256), #13
    'DenseNet121': (256, 256), #14
    'DenseNet169': (256, 256), #15
    'DenseNet201': (256, 256), #16 
    #'NASNetMobile': (224, 224), #17
    #'NASNetLarge': (331, 331), #18 
    'EfficientNetB0': (256, 256), #19
    'EfficientNetB1': (256, 256), #20
    'EfficientNetB2': (256, 256), #21
    'EfficientNetB3': (256, 256), #22
    'EfficientNetB4': (256, 256), #23
    'EfficientNetB5': (256, 256), #24
    'EfficientNetB6': (256, 256), #25
    'EfficientNetB7': (256, 256), #26
    # Add sizes for EfficientNetV2 models if needed
}

model_info = {
    'Xception': (Xception, preprocess_xception),
    'VGG16': (VGG16, preprocess_vgg16),
    'VGG19': (VGG19, preprocess_vgg19), #3
    'ResNet50': (ResNet50, preprocess_resnet), #4
    'ResNet50V2': (ResNet50V2, preprocess_resnet_v2), #5
    'ResNet101': (ResNet101, preprocess_resnet), #6
    'ResNet101V2': (ResNet101V2, preprocess_resnet_v2), #7
    'ResNet152': (ResNet152, preprocess_resnet), #8
    'ResNet152V2': (ResNet152V2, preprocess_resnet_v2), #9
    'InceptionV3': (InceptionV3, preprocess_inception_v3), #10
    'InceptionResNetV2': (InceptionResNetV2, preprocess_inception_resnet_v2), #11
    'MobileNet': (MobileNet, preprocess_mobilenet), #12
    'MobileNetV2': (MobileNetV2, preprocess_mobilenet_v2), #13
    'DenseNet121': (DenseNet121, preprocess_densenet), #14
    'DenseNet169': (DenseNet169, preprocess_densenet), #15
    'DenseNet201': (DenseNet201, preprocess_densenet), #16
    #'NASNetMobile': (NASNetMobile, preprocess_nasnet), #17
    #'NASNetLarge': (NASNetLarge, preprocess_nasnet), #18
    'EfficientNetB0': (EfficientNetB0, preprocess_efficientnet), #19
    'EfficientNetB1': (EfficientNetB1, preprocess_efficientnet), #20
    'EfficientNetB2': (EfficientNetB2, preprocess_efficientnet), #21
    'EfficientNetB3': (EfficientNetB3, preprocess_efficientnet), #22
    'EfficientNetB4': (EfficientNetB4, preprocess_efficientnet), #23
    'EfficientNetB5': (EfficientNetB5, preprocess_efficientnet), #24
    'EfficientNetB6': (EfficientNetB6, preprocess_efficientnet), #25
    'EfficientNetB7': (EfficientNetB7, preprocess_efficientnet), #26
}

# Função para preprocessar imagem
def preprocess_image(img_path, model_name):
    target_size = model_input_sizes[model_name]
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    preprocess_input_func = model_info[model_name][1]
    return preprocess_input_func(img_array_expanded_dims)

# Inicializar modelo
def init_model(model_name):
    model_class = model_info[model_name][0]
    model = model_class(weights='imagenet', include_top=False, pooling='avg')
    return model

# Extrair características
def extract_features(model, img_path, model_name):
    preprocessed_img = preprocess_image(img_path, model_name)
    features = model.predict(preprocessed_img)
    return features.flatten()

# Função para executar K-Fold de forma estratificada
def run_stratified_kfold_regressions(X, y, group_labels, model_cnn):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    models = [
        ('Linear Regression', LinearRegression()),
        ('Random Forest', RandomForestRegressor(random_state=42)),
        ('SVR', SVR()),
        ('Lasso', Lasso(random_state=42)),
        ('ElasticNet', ElasticNet(random_state=42)),
        ('Ridge', Ridge(random_state=42))
    ]
    
    all_results = []
    
    for name, model in models:
        mse_scores, rmse_scores, mae_scores, r2_scores, r_scores = [], [], [], [], []
        print(f"\nRegressão com {name}:")
        fold_number = 1
        for train_index, test_index in skf.split(X, group_labels):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            # Verificar a distribuição de C1 e C2 em cada fold
            train_labels = group_labels[train_index]
            test_labels = group_labels[test_index]
            print(f"Fold {fold_number}:")
            print(f" - Treino: {sum(train_labels == 'C1')} C1, {sum(train_labels == 'C2')} C2")
            print(f" - Teste: {sum(test_labels == 'C1')} C1, {sum(test_labels == 'C2')} C2")
            
            fold_number += 1

            # Treinar o modelo e fazer previsões
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            r = pearsonr(y_test, y_pred)[0] if np.var(y_pred) > 0 else 0
            
            mse_scores.append(mse)
            rmse_scores.append(rmse)
            mae_scores.append(mae)
            r2_scores.append(r2)
            r_scores.append(r)

            for real, pred in zip(y_test, y_pred):
                print(f"Peso Real: {real}, Peso Predito: {pred:.2f}")

            print(f"Fold: {len(mse_scores)}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}, R: {r:.2f}")

        print(f"\nMédia dos resultados para {name}:")
        print(f"MSE: {np.mean(mse_scores):.2f} ± {np.std(mse_scores):.2f}")
        print(f"RMSE: {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
        print(f"MAE: {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
        print(f"R2: {np.mean(r2_scores):.2f} ± {np.std(r2_scores):.2f}")
        print(f"R: {np.mean(r_scores):.2f} ± {np.std(r_scores):.2f}")

        all_results.append({
            'REGRESSAO': name,
            'MSE': f"{np.mean(mse_scores):.2f} ± {np.std(mse_scores):.2f}",
            'RMSE': f"{np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}",
            'MAE': f"{np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}",
            'R2': f"{np.mean(r2_scores):.2f} ± {np.std(r2_scores):.2f}",
            'R': f"{np.mean(r_scores):.2f} ± {np.std(r_scores):.2f}",
            'CNN': model_cnn
        })
        
    return all_results

# Carregar dados
df = pd.read_csv('all_animal_images.csv')

# Adicionar uma nova coluna que identifica se o animal é do grupo C1 ou C2
df['Group'] = df['ID Animal'].apply(lambda x: 'C1' if 'C1_' in x else 'C2')

# Processar imagens e características para cada modelo
final_results = []
for model_name in model_info.keys():
    print(f"\nIniciando processamento com modelo: {model_name}")
    model = init_model(model_name)
    feature_dict = {}
    
    for index, row in df.iterrows():
        animal_id = row['ID Animal']
        img_path = row['Path']
        features = extract_features(model, img_path, model_name)
        if animal_id not in feature_dict:
            feature_dict[animal_id] = []
        feature_dict[animal_id].append(features)
        print(f"Características extraídas para ID {animal_id} da imagem {img_path}")

    for animal_id, feats in feature_dict.items():
        aggregate_features = np.mean(feats, axis=0)
        feature_dict[animal_id] = aggregate_features
        print(f"Vetor de características agregado para ID {animal_id}: {aggregate_features.shape}")

    feature_df = pd.DataFrame.from_dict(feature_dict, orient='index')
    weights = df[['ID Animal', 'Peso', 'Group']].drop_duplicates().set_index('ID Animal')
    feature_df = feature_df.join(weights)

    X = feature_df.iloc[:, :-2].values
    y = feature_df['Peso'].values
    group_labels = feature_df['Group'].values

    results = run_stratified_kfold_regressions(X, y, group_labels, model_name)
    final_results.extend(results)

# Criar DataFrame e salvar em Excel
results_df = pd.DataFrame(final_results)
results_df.to_excel('regression_results_256_padding_seg_alci_will_stratified.xlsx', index=False)
print("Resultados salvos em 'regression_results_256_padding_seg_alci_will_stratified.xlsx'.")


Iniciando processamento com modelo: Xception
1/1 [==============================] - 3s 3s/step
Características extraídas para ID C2_01 da imagem base_alci_auto_seg\C2_01\roi_image_3851_left_bottom_png.rf.1e10cd3c3687b0bde4232dadd025e19d.jpg
1/1 [==============================] - 0s 24ms/step
Características extraídas para ID C2_01 da imagem base_alci_auto_seg\C2_01\roi_image_3852_left_bottom_png.rf.09aa6b3620adc9bb43686a41a331e42f.jpg
1/1 [==============================] - 0s 29ms/step
Características extraídas para ID C2_01 da imagem base_alci_auto_seg\C2_01\roi_image_3853_left_bottom_png.rf.9b91d1e91dd82065536609663734de31.jpg
1/1 [==============================] - 0s 24ms/step
Características extraídas para ID C2_01 da imagem base_alci_auto_seg\C2_01\roi_image_3854_left_bottom_png.rf.675ab759862e944ded5d8d9de3a5783e.jpg
1/1 [==============================] - 0s 30ms/step
Características extraídas para ID C2_02 da imagem base_alci_auto_seg\C2_02\roi_image_9630_left_bottom_png.rf.05c

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 36ms/step
Características extraídas para ID C2_31 da imagem base_alci_auto_seg\C2_31\roi_image_0518_left_bottom_png.rf.6beb4663665d7be0b30377f7d2bc0124.jpg
1/1 [==============================] - 0s 33ms/step
Características extraídas para ID C2_31 da imagem base_alci_auto_seg\C2_31\roi_image_0519_left_bottom_png.rf.2d20ff045b8225184d76fbae1f2d25e5.jpg
1/1 [==============================] - 0s 51ms/step
Características extraídas para ID C2_31 da imagem base_alci_auto_seg\C2_31\roi_image_0520_left_bottom_png.rf.b838dea9a17d998463fc767d216ca8aa.jpg
1/1 [==============================] - 0s 39ms/step
Características extraídas para ID C2_31 da imagem base_alci_auto_seg\C2_31\roi_image_0523_left_bottom_png.rf.31f510a2b91ff34d362d018db6ee5865.jpg
1/1 [==============================] - 0s 40ms/step
Características extraídas para ID C2_31 da imagem base_alci_auto_seg\C2_31\roi_image_0524_left_bottom_png.rf.15914ba526281e50caaba3f76daa960b.jpg
1/1 [=====

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Características extraídas para ID C2_24 da imagem base_alci_auto_seg\C2_24\roi_image_0883_left_bottom_png.rf.cbabc8f51a7490c41e43cc8698ca187b.jpg
1/1 [==============================] - 0s 61ms/step
Características extraídas para ID C2_24 da imagem base_alci_auto_seg\C2_24\roi_image_0884_left_bottom_png.rf.1aef8805aa34b11154db21747d5e520e.jpg
1/1 [==============================] - 0s 56ms/step
Características extraídas para ID C2_24 da imagem base_alci_auto_seg\C2_24\roi_image_0885_left_bottom_png.rf.75f19e8b3d24e5783dd691f25657144c.jpg
1/1 [==============================] - 0s 60ms/step
Características extraídas para ID C2_24 da imagem base_alci_auto_seg\C2_24\roi_image_0886_left_bottom_png.rf.1608245881edbd2e51f7cacf11273649.jpg
1/1 [==============================] - 0s 57ms/step
Características extraídas para ID C2_24 da imagem base_alci_auto_seg\C2_24\roi_image_0887_left_bottom_png.rf.67e62c3704b4ae705f7a7bf7e2d0d6d4.jpg
1/1 [==============================] - 0s 54ms/step
Caracterís